In [31]:
import pandas as pd
import numpy as np
import statistics
import requests
import json
import operator

## Common Methods

In [3]:
def callApi(method, stock,isEarnings = False ):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last = "/10"
    period = "period=annual"
    if isEarnings:
        url = host + ticker + api_method + last + query + period + add + key 
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [4]:
def addTTMToYearLists(method,lists,ttmList):
    reportDate = "reportDate"
    if method == "earnings":
        reportDate = "EPSReportDate"
    if not (method in ttmList.keys()):
        return lists
    first = lists[method][0]
    ttm = ttmList[method][0]
    if first[reportDate] != ttm[reportDate]: 
        lists[method].insert(0, ttmList[method][0])
    return lists

## Ratio Methods

In [5]:
def peRatio(quote):
    if "peRatio" in quote.keys() and quote["peRatio"] is not None:
        return float(quote["peRatio"])
    return 0.0

In [6]:
def priceToBook(stats):
    if "priceToBook" in stats.keys() and stats["priceToBook"] is not None:
        return float(stats["priceToBook"])
    return 0.0

In [7]:
def debtToEquity(stats):
    if "debtToEquity" in stats.keys() and stats["debtToEquity"] is not None:
        return float(stats["debtToEquity"])
    return 0.0 

In [8]:
def pegRatio(stats):
    if stats["pegRatio"] is not None:
        return float(stats["pegRatio"])
    return 0.0

In [9]:
def sharesoutstanding(stats):
    if stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    return 0.0

In [22]:
def marketCap(quote):
    if "marketCap" in quote.keys() and quote["marketCap"] is not None:
        return quote["marketCap"]
    return 0 

In [10]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if not("balancesheet" in balance_sheet.keys()) or len(balance_sheet["balancesheet"]) == 0:
        return 0.0
    latest_balance_sheet = balance_sheet["balancesheet"][0]
    if not("totalAssets" in latest_balance_sheet.keys()) and not("totalLiabilities" in latest_balance_sheet.keys()):
        return 0.0
    return float(latest_balance_sheet["totalAssets"] / latest_balance_sheet["totalLiabilities"])
    

In [11]:
def checkErningProgression(earnings, earning):
    earnings = addTTMToYearLists("earnings", earnings, earning)
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return False
    earningsSet = earnings["earnings"][:-1]
    yearAgoChangePercent = np.array([])
    for earning in earningsSet:
        if not("yearAgoChangePercent" in earning.keys()):
            return False
        yearAgoChangePercent = np.append(yearAgoChangePercent, earning["yearAgoChangePercent"])
    isDecending = (yearAgoChangePercent == np.sort(yearAgoChangePercent)[::-1]).all()
    return isDecending

In [12]:
def sharesOutstanding(stats):
    if stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    return 0.0

In [37]:
def NetCurrentAssetValuePerShare(balanceSheet, stats, price):
    #Net Current Asset Value = Current Assets – (Current Liabilities + Long-Term Liabilities) 
    shares_Outstanding = sharesOutstanding(stats)
    if "balancesheet" in balance_sheet.keys() and len(balance_sheet["balancesheet"]) > 0 and shares_Outstanding !=0:
        latest_balance_sheet = balance_sheet["balancesheet"][0]
        if "currentAssets" in latest_balance_sheet.keys() and "totalLiabilities" in latest_balance_sheet.keys() and "minorityInterest" in latest_balance_sheet.keys():
            netCurrentAssetValue = (latest_balance_sheet["currentAssets"] - latest_balance_sheet["totalLiabilities"] - latest_balance_sheet["minorityInterest"]) / shares_Outstanding
            return price/netCurrentAssetValue
    return 0

In [43]:
tickers = ['DAL', 'NKE']
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    
    quote = callApi("quote", ticker)
    # Less than 10 million don't comsider
    market_cap =  marketCap(quote)
    if market_cap < 10000000:
        continue
    
    # Major data set 
    stats = callApi("advanced-stats", ticker)
    earnings = callApi("earnings", ticker, True)
    earning = callApi("earnings", ticker)
    price = callApi("price", ticker)
    balance_sheet = callApi("balance-sheet", ticker)
        
    data["Ticker"] = ticker
    data ["company Name"] = quote["companyName"]
    data["Price"] = price
    data["Market Cap"] = market_cap
    data["52 High"] = quote["week52High"]
    data["52 Low"] = quote["week52Low"]
    
    # PE Ratio < 7
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 7"] = pe_ratio
    else: 
        data["PE Ratio < 7"] = "NA"
    if pe_ratio < 7 and pe_ratio > 0: 
        points += 1
        
    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1"] = price_to_book
    else: 
        data["Price To Book < 1"] = "NA"
    
    if  price_to_book < 1 and  price_to_book > 0: 
        points += 1
    
    # PEG Ratio 
    peg_ratio = pegRatio(stats)
    if peg_ratio != 0: 
        data["PEG Ratio < 1"] = peg_ratio
    else: 
        data["PEG Ratio < 1"] = "NA"
    if peg_ratio < 1.1 and peg_ratio > 0:
        points += 1
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["Debt to Equity < 0.5"] = debt_Equity
    else: 
        data["Debt to Equity < 0.5"] = "NA"
    if debt_Equity < 0.5 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["Current Ratio > 3"] = current_ratio
    else: 
        data["Current Ratio > 3"] = "NA"
        
    if current_ratio > 3:
        points += 1
    
    # Earning Progression Last 4 year 
    isProgressing = checkErningProgression(earnings, earning)
    data["Earning Progression"] =  isProgressing
    if isProgressing:
        points += 1
    
    # Net Current Asset Value Per Share
    # Graham instituted a rule in which he would only consider buying a stock if its current price was trading at lower than 66% of its NCAVPS.
    NCAVPS =NetCurrentAssetValuePerShare(balance_sheet, stats, price)
    data["NCAVPS <50%"] =  round(NCAVPS,3)
    if NCAVPS < .50 and NCAVPS > 0:
        points += 1
    
    data["Points(6)"] = points

    dataSet.append(data.copy())
dataSet.sort(key=operator.itemgetter("Points(6)"), reverse=True)
df = pd.DataFrame(dataSet)  
df.to_csv("Advance-reserch.csv", sep='\t', encoding='utf-8')
pd.DataFrame(dataSet) 

,Ticker,company Name,Price,Market Cap,52 High,52 Low,PE Ratio < 7,Price To Book < 1,PEG Ratio < 1,Debt to Equity < 0.5,Current Ratio > 3,Earning Progression,NCAVPS <50%,Points(6)
0,DAL,"Delta Air Lines, Inc.",32.82,20934466740,62.48,17.51,-5.64,1.363098,0.02,1.12,1.136698,False,-0.469,1
1,NKE,"NIKE, Inc.",119.28,186061293600,119.93,60.00,73.37,18.434276,-0.40,1.62,1.345901,False,-54.371,0
